Нужно предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

 

Объективные признаки:

 - Возраст
 - Рост
 - Вес
 - Пол
 

Результаты измерения:

 - Артериальное давление верхнее и нижнее
 - Холестерин
 - Глюкоза
 

Субъективные признаки:

 - Курение
 - Употребление Алкоголя
 - Физическая активность
 

Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Таргет - наличие сердечно-сосудистых заболеваний (ССЗ)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score

In [2]:
df = pd.read_csv('train_case2.csv', sep=";")
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
X = df.drop('cardio', axis=1)
X.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,0,18393,2,168,62.0,110,80,1,1,0,0,1
1,1,20228,1,156,85.0,140,90,3,1,0,0,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0


In [4]:
y = df['cardio']
y.head(3)

0    0
1    1
2    1
Name: cardio, dtype: int64

In [5]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [7]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 0.09144178,  0.44594423,  0.05752649, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.46216539,  0.20194394, -0.84752919, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.5702846 , -1.01805752,  1.79801819, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67238553,  0.56794437, -0.01209318, ...,  0.        ,
         0.        ,  1.        ],
       [-0.50009534,  0.32394408, -0.29057185, ...,  0.        ,
         0.        ,  0.        ],
       [-1.76136594,  1.54394554, -0.70828985, ...,  0.        ,
         0.        ,  1.        ]])

Будем обучать этот датасет на разных моделях и записывать метрики по результатам

In [8]:
#Сюда будем записывать метрики для каждой модели

precision_s = []
recall_s = []
auc_s = []
f_score_s = []

Обучим на логистической регрессии

In [9]:
from sklearn.linear_model import LogisticRegression

classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7821358976691873+-0.009748562688833385


Посчитаем метрики

In [10]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

auc = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])

precision_s.append(precision[ix])
recall_s.append(recall[ix])
auc_s.append(auc)
f_score_s.append(fscore[ix])

Обучим на дереве решений

In [11]:
from sklearn.tree import DecisionTreeClassifier

treeClassifier = Pipeline([
    ('features',feats),
    ('tree_classifier', DecisionTreeClassifier(random_state = 42)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(treeClassifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
treeClassifier.fit(X_train, y_train)
y_score = treeClassifier.predict_proba(X_test)[:, 1]

CV score is 0.6334762269496221+-0.007349149606344275


Посчитаем метрики

In [12]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

auc = roc_auc_score(y_true=y_test, y_score=treeClassifier.predict_proba(X_test)[:,1])

precision_s.append(precision[ix])
recall_s.append(recall[ix])
auc_s.append(auc)
f_score_s.append(fscore[ix])

Обучим на Адабусте

In [13]:
from sklearn.ensemble import AdaBoostClassifier

adaBoost = Pipeline([
    ('features',feats),
    ('adaBoost', AdaBoostClassifier(n_estimators=50, learning_rate = 1)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(adaBoost, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

#обучим пайплайн на всем тренировочном датасете
adaBoost.fit(X_train, y_train)
y_score = adaBoost.predict_proba(X_test)[:, 1]

CV score is 0.7950645544058541+-0.008575961406943446


Посчитаем метрики

In [14]:
b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)

auc = roc_auc_score(y_true=y_test, y_score=adaBoost.predict_proba(X_test)[:,1])

precision_s.append(precision[ix])
recall_s.append(recall[ix])
auc_s.append(auc)
f_score_s.append(fscore[ix])

Теперь сведем все полученные метрики в одну таблицу

In [15]:
metrics = pd.DataFrame({"precision": precision_s, "recall": recall_s, "auc": auc_s, "f_score": f_score_s}, 
                      index=["LogisticRegression", "DecisionTreeClassifier", "AdaBoostClassifier"])

metrics

,precision,recall,auc,f_score
LogisticRegression,0.655534,0.837494,0.785239,0.735426
DecisionTreeClassifier,0.503543,1.000000,0.636365,0.669808
AdaBoostClassifier,0.714960,0.773377,0.797929,0.743022


Анализируя таблицу можно сделать вывод что бустинг получил более менее уравновешенные метрики. Остальные модели завышали recall.
Я считаю что бустинг справился лучше, а хуже всех справилось дерево решений, слишком большой разброс между precision и recall